In [12]:
## 安装调用大模型的openai库和读pdf的pypdf库

# !pip install openai pypdf

In [ ]:

import openai
import json

# API密钥
API_TOKEN = ""  # 此处需要填写对应的api密钥
MODEL_NAME = 'deepseek-v3'  # 选择要调用的模型名称

# 请求URL
BASE_URL = "https://ng.115.zone/v1/"

# 创建一个LLM顾问实例对象
llm_client = openai.OpenAI(api_key=API_TOKEN, base_url=BASE_URL)


# 不要修改这个函数
# 此函数的作用是发送LLM调用请求
def llm_generate(messages: list[dict[str, str]], model_name=MODEL_NAME):
    """
    发送OpenAI格式的大模型请求

    参数:
    - messages: 包含对话历史的字典列表，每个字典包含role和content键,role为user或assistant
    - model_name: 要调用的模型名称

    返回:
    - 模型生成的回复内容
    """
    try:
        # 发送请求
        response = llm_client.chat.completions.create(
            model=model_name,
            messages=messages,
            stream=False
        )

        # 返回模型生成的内容
        return response.choices[0].message.content

    except Exception as e:
        print(f"请求发生错误: {e}")
        return None

In [14]:
# 简单尝试提问的问题
question = "西安电子科技大学新手入学实验班选拔考试考哪些科目，所占的比重如何"

print("## 问题：", question)
## 模型推理代码⬇️

# 表明身份是用户user,内容是对应的问题
messages = [
    {"role": "user", "content": question},
]

# 调用llm_generate来生成内容并输出
content = llm_generate(messages)
print(content)

## 问题： 西安电子科技大学新手入学实验班选拔考试考哪些科目，所占的比重如何
西安电子科技大学（XDU）的新生实验班（如钱学森空间科学实验班、图灵人工智能实验班等）选拔考试通常涵盖多个科目，具体科目和比重可能因年度和实验班类型有所调整。以下是基于往年信息的整理和一般情况说明，供参考：

---

### **一、常见考试科目及比重**
1. **数学**  
   - **比重**：约30%-40%  
   - **内容**：高中数学知识为主，可能涉及微积分、线性代数等大学先修内容（尤其理科实验班）。题型包括选择、填空、解答题，难度高于高考。

2. **物理/理科综合**  
   - **比重**：20%-30%（理工科实验班常见）  
   - **内容**：力学、电磁学为主，部分题目可能涉及竞赛难度。

3. **英语**  
   - **比重**：15%-20%  
   - **内容**：阅读、完形填空、写作，部分班级可能要求听力，词汇量要求可能达到四级水平。

4. **计算机基础/编程**（部分实验班如人工智能、计算机类）  
   - **比重**：15%-25%  
   - **内容**：基础编程逻辑（如C/Python）、算法思维，可能有简单代码题或选择题。

5. **综合素质测试**  
   - **比重**：10%-20%  
   - **内容**：逻辑推理、心理测试或简短面试，考察学习潜力和适应能力。

---

### **二、其他可能的选拔环节**
- **面试**：部分实验班在笔试后增加面试，考察表达能力、专业兴趣和科研潜力。  
- **竞赛加分**：数学、物理、信息学竞赛获奖者可能在选拔中获得额外权重（具体政策需以当年通知为准）。

---

### **三、注意事项**
1. **每年动态调整**：各实验班的选拔科目和比重可能根据培养目标调整，例如：  
   - **钱学森班**：侧重数学、物理和工程思维。  
   - **图灵班**：可能加强编程和算法测试。  
2. **官方渠道**：录取后学校会发布详细选拔通知，建议关注**西电教务处官网**或加入新生群获取最新文件。  
3. **备考建议**：  
   - 复习高中数学、物理核心知识点，适当接触竞赛题。  
   - 提前了解编程基础（如Pytho

---

# 开始合成数据

## 准备训练文件
在data目录中我们寻找了若干文件对其文本进行读取操作，并且进行了简单的清洗

### 将PDF政策文件提取为TXT

In [15]:
## 读取文件
from pathlib import Path
from docx import Document
from pypdf import PdfReader

# 项目 data 文件夹路径
data_folder = Path("data2")
output_file = "output2.txt"

def read_docx(file_path):
    """读取 Word 文档内容"""
    doc = Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

def read_pdf(file_path):
    """读取 PDF 文档内容"""
    text = []
    reader = PdfReader(file_path)
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:  # 避免空页导致 None
            text.append(page_text)
    return "\n".join(text)

def main():
    all_texts = []

    # 遍历 data 文件夹下所有文件（非递归）
    for file in sorted(data_folder.iterdir()):
        if file.suffix.lower() == ".docx":
            print(f"读取 Word 文件: {file.name}")
            all_texts.append(read_docx(file))
        elif file.suffix.lower() == ".pdf":
            print(f"读取 PDF 文件: {file.name}")
            all_texts.append(read_pdf(file))
        else:
            print(f"跳过文件: {file.name}（非 docx/pdf）")

    # 保存到 output.txt
    with open(output_file, "w", encoding="utf-8") as f:
        f.write("\n\n".join(all_texts))

    print(f"所有文档内容已保存到 {output_file}")

if __name__ == "__main__":
    main()

读取 PDF 文件: 体检收费标准.pdf
读取 Word 文件: 关于试点班的参考建议.docx
读取 Word 文件: 可选志愿.docx
读取 Word 文件: 实验班计划选拔名额.docx
读取 Word 文件: 新生常见问题答疑.docx
读取 PDF 文件: 新生组织团关系转入.pdf
跳过文件: 校园网套餐 - 摘自校园网自助服务平台.png（非 docx/pdf）
读取 PDF 文件: 综合楼及宿舍介绍.pdf
读取 PDF 文件: 缴费指南.pdf
所有文档内容已保存到 output2.txt


### 对txt文件数据进行简单处理

In [16]:
import re

input_file = "output2.txt"            # 输入文本文件
output_file = "output_cleaned2.txt"   # 输出文本文件

# 匹配页尾标注：中文破折号 — 数字 — 或 英文破折号 - 数字 -
page_mark_pattern = re.compile(r"^(?:—|-)\s*\d+\s*(?:—|-)$")

def clean_text(file_path):
    cleaned_text = []

    with open(file_path, "r", encoding="utf-8") as f:
        # 去掉空行和首尾空格
        lines = [line.strip() for line in f if line.strip()]

    for line in lines:
        # 删除页尾标注
        if page_mark_pattern.match(line):
            continue
        cleaned_text.append(line)

    # 合并所有行为一行
    return "".join(cleaned_text)

def main():
    cleaned_text = clean_text(input_file)
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(cleaned_text)

    print(f"处理完成，结果已保存到 {output_file}")

if __name__ == "__main__":
    main()


处理完成，结果已保存到 output_cleaned2.txt


In [17]:
with open("output_cleaned2.txt", "r", encoding="utf-8") as f:
    text = f.read()
print(len(text), "characters")

9417 characters


## 构建一个随机抽取函数

从导出的TXT中随机抽取一段文字（默认为抽取200字）

In [18]:
import random

# 一个阅读pdf文字并且转pdf的函数
# 随机抽取文段，以此来让大模型根据这个文段进行提问

def random_text_chunk(txt_path, chunk_length=800):
    """
    从指定txt文件中随机读取一个长度为 chunk_length 个汉字的文本片段。
    如果文本长度不足，则直接返回全文。
    """
    with open(txt_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # 如果文本不够长，直接返回全部内容
    if len(text) <= chunk_length:
        return text

    # 否则随机选取一个起始位置
    start = random.randint(0, len(text) - chunk_length)
    chunk = text[start:start + chunk_length]

    return chunk


text = random_text_chunk("output_cleaned2.txt")
print(text)

让他考你几个专业问题，看看你是不是真的了解自己想去的专业并对它感兴趣，重新考虑问题1。还喜欢那就留守吧解释：1.我当年高考填志愿时候充分研究了微电子专业，但是还是进了西电后两个月就后悔没去计算机。我发现我对哪个专业都没有特别的兴趣也没有特别的抵触。(所以我倒也愿意研究生继续学微电子……)2.计算机类再就业寒冬，它还是就业最好的大类3.在普通班可以大类分流，而电信类特点是每个院不同专业学的差不多、就业差不多，你大可以分流去一个小专业，获得好看的排名，有利于保研、保外(出国)、甚至可能就业上也占便宜。4.在普通班还有大二升大三转专业的机会，在试点班就没了。附：西电最后一次公布各学院毕业生平均工资(2019年毕业的）↓《其它常见问题》1.卓越班推荐吗？①卓越班因为可以退班，所以大概率退班会退到卓越班保研线低于对应普通班保研线。但是，从22级起，一些卓越班就连大学物理这种基础课都单独出更难的卷子，所以退到普通班比起原本在普通班的吃亏。②卓越班因为研究生少一年，所以一方面可以早一年工作赚钱，另一方面优的导师会因此不想要卓越班的学生。综上，自己考虑吧。1.卓越的退出机制。卓越在大二下和大三下会组织分流，其中均分没有达到80的会被强制分流退出，也可以主动退出。大二下那次退出会进入普通班，此时就是正常的普通班学生，可以保外，所以，每年卓越都会有大牛选择主动退出从而保外校。但大三那次退出进入普通班就会丧失所有保研资格。2.卓越保研机制。卓越班只能保本校而且只能保内，不能跨院，保研率50%是按入班人数算的，固定15个。如果当届卓越班退出人数多，以19级电卓为例，最后只有16人，15个保研名额就是闭眼保研。那一届电卓82分就保了，而普通班保研需要88分。但是！并不是每一届都像19级那么幸运，如果退出的人少了，保研难度甚至会大于普通班，有赌的成分。3.卓越的教学资源。除了政治之类的课，都是小班教学


## 让模型根据文本提问

In [19]:
## 用于提问题的提示词⬇️

# 问答前提，可以自定义prompt文本

prompt = "根据下面的内容提取新生可能进行提问的重要信息，然后以新生的口吻提一个问题，注意该问题的答案必须在我提供的片段当中，注意你只返回问题，不要返回任何额外的信息或回答"

# llm是作为system角色回答的
## 模型推理代码⬇️
messages = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": "以下是学校规定或者通知要求：\n\n" + text},
]
content = llm_generate(messages)
print(content)

在普通班和卓越班之间选择时，关于保研机会有哪些主要区别需要考虑？


## 构建模型提问函数

为了提取出文字，我们使用input长度来砍掉用于模型生成的文本段

In [20]:

def question_model_generate(text):
    """
    根据提供的PDF文本内容（如校规），调用模型生成一个以学生口吻提出的问题，
    该问题的答案必须能从提供的文本中找到。

    参数:
    - text (str): 从PDF中提取的文本内容，作为生成问题的依据。

    返回:
    - str: 模型根据参考文本生成的学生提问。
    """
    messages = [
        {"role": "system", "content": "请以西电新生的口吻提出一个自然合理，符合逻辑的问题，问题的答案必须能在我所提供的校规片段中找到。不要包含情景描述描述，例如自称我刚入学，对别人称呼学长等，只输出一个问题，不输出解释或答案。"},
        {"role": "user", "content": text},
    ]
    content = llm_generate(messages)
    return content


# 测试函数是否正常工作
question = question_model_generate(text)
print(question)

卓越班如果在大三下学期退出，会有什么后果？


## 构建模型回答函数

In [21]:
# 根据提出的问题生成回答

def answer_model_generate(question, text):
    """
    根据用户提出的问题和提供的PDF文本内容（如校规），调用模型生成回答。

    参数:
    - question (str): 用户提出的问题。
    - text (str): 从PDF中提取的文本内容，作为回答问题的依据。

    返回:
    - str: 模型根据问题和参考文本生成的回答。
    """
    # 这里注意，要将参考文本输入到system prompt当中
    messages = [
        {"role": "system",
         "content": "你是西电新生百事通助手，请根据学生提出的单个问题，以及下面我提供的校规或公告内容，给出准确、完整的回答。回答必须严格依据校规内容，不加入校外信息。只输出回答内容，不输出提问或额外说明，生成的同时请你再次检查是否符合我的要求以及问题是否合理。\n\n" + text},
        {"role": "user", "content": question},
    ]
    content = llm_generate(messages)
    return content


# 测试函数是否正常工作
answer = answer_model_generate(question, text)
print(answer)

卓越班如果在大三下学期退出，将进入普通班并丧失所有保研资格。


## 开始通过循环生成数据

In [22]:
import time
question_num = 120  # 控制生成的数据数量

GENERATE_QA_DATA = []
for i in range(question_num):
    print(f"############# 生成第{i}个问题")
    # 随机抽取一段文本
    text = random_text_chunk("output_cleaned2.txt")
    # print(f"#### 抽取的文本\n{text}")
    # 模型生成一个问题
    question = question_model_generate(text)
    print("### 问题：")
    print(question)
    # 模型再给一个答案
    answer = answer_model_generate(question, text)
    print("### 回答：")
    print(answer)
    time.sleep(1.5)
    GENERATE_QA_DATA.append((question, answer))

############# 生成第0个问题
### 问题：
卓越班和普通班在保研外校时的成绩排名和分数是如何比较的？
### 回答：
最适合保外的班有且只有普通班。设想下面两个人(甲考的卷子还更难)：
甲——专业排名：4/30，平均分91
乙——专业排名：15/230，平均分90.5
外校老师通常会选择排名更靠前的普通班学生。部分高校(如浙大)会核查特殊班级身份，但大多数学校不会。卓越班可能面临更难基础课卷子，导致成绩排名和分数对比普通班处于劣势。
############# 生成第1个问题
### 问题：
卓越班学生在分流时如果选择主动退出，具体有哪些退出时间节点及其对应的保研资格影响？
### 回答：
卓越班学生可以在大二下和大三下两个时间节点选择主动退出。具体影响如下：

1. 大二下学期退出：将转入普通班，此时作为普通班学生享有正常的保研资格，可以保外校。

2. 大三下学期退出：同样会转入普通班，但会丧失所有保研资格。

需要注意的是，卓越班的退出机制还包括强制分流：如果均分未达到80分，学生将被强制分流退出，退出时间节点和上述主动退出相同，对应的保研资格影响也一致。
############# 生成第2个问题
### 问题：
如何查询不同实验班的年度考核或滚动分流的具体规则？
### 回答：
关于实验班年度考核/滚动分流的规则，需特别注意以下要点：

1. 所有实验班均实行"滚动分流"或"年度考核"机制，具体执行标准由各实验班自行制定，建议入学后及时向辅导员或班级导师索取本班的《培养方案实施细则》。

2. 重点班型特殊规则：
- 计算机拔尖班/钱学森班等5个本博贯通班：实行"末位警示"，连续两年专业排名后10%需参加分流考核
- 卓越班：学年均分低于80分或单科挂科即触发分流评估
- 特色班（微电子/密码等）：通常要求前三年保持专业前50%排名

3. 分流时间节点统一为每学年9月，与大类专业分流同步进行。分流结果公示后，未达标者将转回对应大类普通班。

（注：具体条款可能因年度调整，请以教务处最新版《实验班管理办法》为准）
############# 生成第3个问题
### 问题：
学校综合楼里有哪些奶茶店可以选择？
### 回答：
老综合楼：西电人自己的奶茶店、沪上阿姨、库迪咖啡、瑞幸咖啡

新综合楼：蜜雪冰城、书亦烧仙草、益禾堂、茶百道、茶话弄、

## 以Alpaca数据集格式保存到本地

In [23]:
import json


file_path = 'dataset.json'  # 输出文件路径

# 用列表收集所有数据
data_list = []

for question, answer in GENERATE_QA_DATA:
    alpaca_data = {
        "instruction": question,
        "input": "",
        "output": str(answer)  # 保证 output 为字符串，避免 object 类型报错
    }
    data_list.append(alpaca_data)

# 写入标准 JSON 数组文件，并美化格式
with open(file_path, 'a', encoding='utf-8') as f:
    json.dump(data_list, f, ensure_ascii=False, indent=4)

print(f"数据已保存至 {file_path}")



数据已保存至 dataset.json


## 接下来的作业本

尝试修改上一讲的代码，来微调新的数据集